There is not an issue with the defintion Imani passed for natrual cubic spline interpolation. That is: 


In [1]:
import numpy as np
def ncubic_spline(x, y):
    #pdb.set_trace();
    
    a = y;
    n = len(x) - 1;
    h = np.zeros(n) 
    for i in range(0, n):
        h[i] = x[i+1] - x[i]
        
    alpha = np.zeros(n) 
    # the a values are your y values
    for i in range(1, n):
        alpha[i] = ((3/h[i])*(a[i+1] - a[i])) - ((3/h[i-1])*(a[i] - a[i-1]))
        
    l = np.zeros(n + 1) 
    mu = np.zeros(n + 1)
    z = np.zeros(n + 1) 
    b = np.zeros(n + 1) 
    c = np.zeros(n + 1) 
    d = np.zeros(n + 1)
    #print("d = " + str(d));
    
    l[0] = 1
    mu[0] = 0
    z[0] = 0
    for i in range(1, n):
        l[i] = 2*(x[i+1] - x[i-1]) - h[i-1]*mu[i-1]
        mu[i] = h[i]/l[i]
        z[i] = alpha[i] - (h[i-1]*z[i-1])/l[i]
    
    #Below, Idk if the indexing for l,z,c is correct since size is already n-and we want to insert the values at n 
    l[n] = 1
    z[n] = 0
    c[n] = 0
    #pdb.set_trace()
    for j in np.arange(n-1,-1,-1):
        c[j] = z[j] - mu[j]*c[j+1]
        b[j] = ((y[j+1] - y[j])/h[j]) - (h[j]*(c[j+1] + 2*c[j])/3.0)
        d[j]= (c[j+1] - c[j])/(3*h[j])
        
        print("a = y = " + str(y[j]) + " b = " + str(b[j]) + " c = " + str(c[j]) + " d = " + str(d[j]));
    for i in range(n):
        print('s_{' + str(i) + '}(x)=' + str(y[i]) + ' + (' + str(b[i]) + ')(x-' + str(x[i]) + ')' + ' + (' + str(c[i]) + ')(x-' + str( x[i] ) + ')^2' + ' + (' + str(d[i]) + ')(x-' + str(x[i]) + ")^3"  )
    print("--------------------------------------")
    for i in range(n):
        print( "s_{" +  str(i) + "}(x)=" + str(y[i]) + " + "  + str(b[i]) + "*(x-" + str(x[i])  + ")+" + str(c[i]) + "*(x-" +  str(x[i]) +")**2+" + str(d[i]) + "*(x-" + str(x[i]) + ")**3" );



But rather an issue when passing arguments for x and y(This is TSLA data). That is 

In [2]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 11])
y = np.array([264.769989, 337.320007,350.480011,332.799988,307.019989,319.880005,279.859985,238.690002,185.160004,223.460007,241.610001,225.610001,240.869995])
print("Size of x is: " + str(np.size(x)));
print()
print("Size of y is: " + str(np.size(y)));

Size of x is: 12

Size of y is: 13


Thus, when we use the above arguments in the defintion 

In [3]:
ncubic_spline(x, y)

a = y = 241.610001 b = 27.67624216329896 c = -65.51436324494844 d = 21.83812108164948
a = y = 223.460007 b = 120.18217180165955 c = -120.29108508001505 d = 18.258907278355537
a = y = 185.160004 b = -135.6537229543171 c = 321.07613147148317 d = -147.12240551716607
a = y = 238.690002 b = -69.97487385128923 c = -135.87075195880774 d = 152.31562781009697
a = y = 279.859985 b = -51.89457448948246 c = 84.02226321362755 d = -73.2976717241451
a = y = 319.880005 b = 74.03486796951564 c = -213.09346356108722 d = 99.03857559157159
a = y = 307.019989 b = -33.640562447487206 c = 176.29759945177437 d = -129.7970210042872
a = y = 332.799988 b = -44.90135461753657 c = -59.466766299582304 d = 78.5881219171189
a = y = 350.480011 b = 21.575742735490643 c = -29.15026545344482 d = -10.105500282045828
a = y = 337.320007 b = 136.79840957557406 c = -170.88247563663865 d = 47.244070061064605
a = y = 264.769989 b = 129.51084321221285 c = 0.0 d = -56.96082521221288
s_{0}(x)=264.769989 + (129.51084321221285)(x-0)

$s_{10}(x)$ says for element 10 to 11 we have the following polynomial. 

$s_{10}(x)=241.610001 + (27.67624216329896)(x-10) + (-65.51436324494844)(x-10)^2 + (21.83812108164948)(x-10)^3$ 
for $x_{10} < x < x_{11}$ 

in our case that is x in between months 10 and 11. (i.e. for $10 < x < 11$ ) 

Then the above will not interpolate up until September, instead the last polynomial(the polynomial above) will go until august. 

In [7]:
# To verify we will use the polynomial and match it up the nodes.

def f(x):
    value = 241.610001 + 27.67624216329896*(x-10)+-65.51436324494844*(x-10)**2+21.83812108164948*(x-10)**3
    return value 

If the above polynomial is correct it should give back the closing prices for August 2019 to September 2019. 

In [9]:
print(f(12))
print(f(11))
print(f(10))

209.61000099999998
225.61000100000004
241.610001


Instead it prints the correct closing prices from July to August that is:

In [11]:
print(y[12])
print(y[11])
print(y[10])

240.869995
225.610001
241.610001


Notice that y at element 12 exists, that is because we know the closing price for Septemeber in our data. 

In order to fix the incorrect polynomial we need to extend the vector x. That is 

In [12]:
x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 11, 12])
ncubic_spline(x, y)

a = y = 225.610001 b = -58.96300781607924 c = 111.33450272411883 d = -37.11150090803961
a = y = 241.610001 b = 10.452734651624343 c = -95.34635333949593 d = 68.8936186878716
a = y = 223.460007 b = 124.79719673046127 c = -112.29762742594392 d = 5.65042469548266
a = y = 185.160004 b = -136.89031515784941 c = 318.93429094974607 d = -143.74397279189665
a = y = 238.690002 b = -69.64352996596158 c = -135.29684752593067 d = 151.41037949189226
a = y = 279.859985 b = -51.98335782726064 c = 83.86848600385629 d = -73.05511117659564
a = y = 319.880005 b = 74.05865743530076 c = -213.05225915487927 d = 98.97358171957852
a = y = 307.019989 b = -33.64693697284952 c = 176.28655903671387 d = -129.77960606386438
a = y = 332.799988 b = -44.89964598187244 c = -59.46380904554824 d = 78.5834560274207
a = y = 350.480011 b = 21.575282718196455 c = -29.15105405452057 d = -10.104251663675889
a = y = 337.320007 b = 136.79854100908665 c = -170.8822784863697 d = 47.24374147728304
a = y = 264.769989 b = 129.51077749

In [15]:
def f(x):
    value = 225.610001 + -58.96300781607924*(x-11)+111.33450272411883*(x-11)**2+-37.11150090803961*(x-11)**3
    return value 


We now have $ s_{11}(x)$ which is a polynomial where $ x_{11} < x < x_{12}$ thus we now interpolate all the months and if we go to $x_{13}$ we should interpolate October.

Verifing we have 

In [16]:
print(f(11))
print(f(12))

225.610001
240.86999500000002


Where the actual values are 

In [18]:
print(y[11])
print(y[12])

225.610001
240.869995


Then adding the 12th month to the vector x should fix the issue.

We did a number of things using the incorrect results so that now must be fixed. (We also presented with the wrong data but I doubt he will even notice.)

Below is the interpolated Closing price for the month of october October: 



In [20]:
print(f(13))

256.12998899999997


Looking at the last portion of the code, calculating error: 

In [21]:
print('The absolute error for Tesla rate of return is: ' + str( abs(209.61000099999998 - tesla_rate)  ) );
print();


NameError: name 'tesla_rate' is not defined

I think we calculated it using the interpolated closing price instead of finding the interpolated rate of return. 
We need to use the rate of return formula with the closing price we have found: 

I.e: $ \frac{y[13] - y[12]}{y[12]}$